# 7.3 Data Handling - Transform

#### 데이터 변형을 왜 해야할까?

1) 중복 제거하기

2) 데이터 형태 맞춰주기

3) 컴퓨터가 인식할 수 있게 하기

...

결국은 정확한 분석을 위해 필수적으로 진행되어야 함!


In [41]:
import pandas as pd
import numpy as np

### 1. Remove Duplicated Data : 중복 값 제거

In [20]:
data = pd.DataFrame({'k1' : ['one'] * 3 + ['two'] * 4,
                  'k2' : [1,1,2,3,3,4,4]})
print(data)

    k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4


In [22]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

### Drop Duplicated data

In [24]:
print(data.drop_duplicates()) ## 중복 데이터 제거(앞의 True인 1,4,6 제거)

    k1  k2
0  one   1
2  one   2
3  two   3
5  two   4


In [29]:
data ['v1'] = range(7) ## 새 칼럼 추가
print(data)
print("============================")
print(data.drop_duplicates(['k1'])) ## k1 칼럼 기준으로 중복제거

    k1  k2  v1
0  one   1   0
1  one   1   1
2  one   2   2
3  two   3   3
4  two   3   4
5  two   4   5
6  two   4   6
    k1  k2  v1
0  one   1   0
3  two   3   3


In [27]:
print(data.drop_duplicates(['k1', 'k2'], keep='last')) # last : 마지막 발견 값 반환

    k1  k2  v1
1  one   1   1
2  one   2   2
4  two   3   4
6  two   4   6


In [33]:
print(data.drop_duplicates(['k1', 'k2']))

    k1  k2  v1
0  one   1   0
2  one   2   2
3  two   3   3
5  two   4   5


### 2. Transform Data using function & maping : 배열 값 기반으로 변형

In [35]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon', 'pastrami',
                             'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.8, 8, 3, 5, 6]})

print(data)

          food  ounces
0        bacon     4.0
1  pulled pork     3.0
2        bacon    12.0
3     Pastrami     6.0
4  corned beef     7.8
5        Bacon     8.0
6     pastrami     3.0
7    honey ham     5.0
8     nova lox     6.0


In [36]:
## 이 고기는 어느 동물한테서 나올까?
meat_to_animal = {
    'bacon' : 'pig',
    'pulled pork' : 'pig',
    'pastrami' : 'cow',
    'corned beef' : 'cow',
    'honey ham' : 'pig',
    'nova lox' : 'salmon'
}

#### map : 리스트에 있는 각각의 값들에 대해 적용, 실행시켜주는 함수

In [37]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
print(data)
#cf) lambda 함수 이용 가능. data['food'].map(lambda x: meat_to_animal[x.lower()])

          food  ounces  animal
0        bacon     4.0     pig
1  pulled pork     3.0     pig
2        bacon    12.0     pig
3     Pastrami     6.0     cow
4  corned beef     7.8     cow
5        Bacon     8.0     pig
6     pastrami     3.0     cow
7    honey ham     5.0     pig
8     nova lox     6.0  salmon


### 3. Substitute values  : 누락된 값에 대한 치환

In [39]:
data = pd.Series([1., -999., 2., -999., -1000., 3.]) ## 보통 결측치는 9로 표현
print(data)

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64


In [42]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [44]:
data.replace([-999, -1000], [np.nan, 0]) ## 여러개

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [45]:
data.replace({-999: np.nan, -1000:0}) ## 딕셔너리 형태

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 4. Change Axis Index name : 바로 축 이름 변경 가능

In [47]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                    index = ["Ohio", "Colorado", "New York"],
                    columns = ['one', 'two', 'three', 'four'])

data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [48]:
data.index = data.index.map(str.upper)
print(data)

          one  two  three  four
OHIO        0    1      2     3
COLORADO    4    5      6     7
NEW YORK    8    9     10    11


In [50]:
print(data.rename(index=str.title, columns = str.upper))

          ONE  TWO  THREE  FOUR
Ohio        0    1      2     3
Colorado    4    5      6     7
New York    8    9     10    11


In [53]:
print(data.rename(index={'OHIO': "INDIANA"},
            columns = {'three': 'peekaboo'}))

          one  two  peekaboo  four
INDIANA     0    1         2     3
COLORADO    4    5         6     7
NEW YORK    8    9        10    11


In [55]:
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace = True)
print(data)

          one  two  three  four
INDIANA     0    1      2     3
COLORADO    4    5      6     7
NEW YORK    8    9     10    11


### 5. Individualization & Quantization : 그룹별 분석

In [56]:
age = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [58]:
bins = [18,25,35,60,100]
cats = pd.cut(age,bins)

print(cats)

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]


In [60]:
### Categorical : 범주형

print(cats.codes)
print("============================")
print(cats.categories)
print("============================")
print(pd.value_counts(cats))

[0 0 0 1 0 0 2 1 3 2 2 1]
IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')
(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64


In [61]:
pd.cut(age, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [62]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(age, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [MiddleAged < Senior < YoungAdult < Youth]

In [63]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.74, 0.99], (0.25, 0.5], (0.5, 0.74], (0.0078, 0.25], (0.74, 0.99], ..., (0.5, 0.74], (0.0078, 0.25], (0.5, 0.74], (0.25, 0.5], (0.0078, 0.25]]
Length: 20
Categories (4, interval[float64]): [(0.0078, 0.25] < (0.25, 0.5] < (0.5, 0.74] < (0.74, 0.99]]

In [65]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
print(cats)
print(pd.value_counts(cats))

[(-0.696, -0.0302], (0.615, 3.314], (-0.696, -0.0302], (-2.747, -0.696], (-2.747, -0.696], ..., (-0.696, -0.0302], (-0.696, -0.0302], (-0.696, -0.0302], (0.615, 3.314], (-0.696, -0.0302]]
Length: 1000
Categories (4, interval[float64]): [(-2.747, -0.696] < (-0.696, -0.0302] < (-0.0302, 0.615] < (0.615, 3.314]]
(0.615, 3.314]       250
(-0.0302, 0.615]     250
(-0.696, -0.0302]    250
(-2.747, -0.696]     250
dtype: int64


### 6. Find outlier

In [69]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000,4))

print(data.describe())

                 0            1            2            3
count  1000.000000  1000.000000  1000.000000  1000.000000
mean     -0.067684     0.067924     0.025598    -0.002298
std       0.998035     0.992106     1.006835     0.996794
min      -3.428254    -3.548824    -3.184377    -3.745356
25%      -0.774890    -0.591841    -0.641675    -0.644144
50%      -0.116401     0.101143     0.002073    -0.013611
75%       0.616366     0.780282     0.680391     0.654328
max       3.366626     2.653656     3.260383     3.927528


In [72]:
col = data[3]
print(col[np.abs(col)>3])
print("============================")
print(data[(np.abs(data)>3).any(1)])

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64
            0         1         2         3
5   -0.539741  0.476985  3.248944 -1.021228
97  -0.774363  0.552936  0.106061  3.927528
102 -0.655054 -0.565230  3.176873  0.959533
305 -2.315555  0.457246 -0.025907 -3.399312
324  0.050188  1.951312  3.260383  0.963301
400  0.146326  0.508391 -0.196713 -3.745356
499 -0.293333 -0.242459 -3.056990  1.918403
523 -3.428254 -0.296336 -0.439938 -0.867165
586  0.275144  1.179227 -3.184377  1.369891
808 -0.362528 -3.548824  1.553205 -2.186301
900  3.366626 -2.372214  0.851010  1.332846


In [73]:
data[np.abs(data)>3] = np.sign(data) * 3
print(data.describe())

                 0            1            2            3
count  1000.000000  1000.000000  1000.000000  1000.000000
mean     -0.067623     0.068473     0.025153    -0.002081
std       0.995485     0.990253     1.003977     0.989736
min      -3.000000    -3.000000    -3.000000    -3.000000
25%      -0.774890    -0.591841    -0.641675    -0.644144
50%      -0.116401     0.101143     0.002073    -0.013611
75%       0.616366     0.780282     0.680391     0.654328
max       3.000000     2.653656     3.000000     3.000000


### 7. Substitution and Random Sampling

In [75]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5,4))

sampler = np.random.permutation(5)
print(sampler)

[1 0 2 3 4]


In [77]:
print(df.take(np.random.permutation(len(df))[:3]))

## 이외에도 np.random.randint를 통해 표본 생성이 가능

    0   1   2   3
1   4   5   6   7
3  12  13  14  15
0   0   1   2   3


### 8. Indicator / Dummy Variables

In [79]:
df = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1' : range(6)})

print(pd.get_dummies(df['key']))

   a  b  c
0  0  1  0
1  0  1  0
2  1  0  0
3  0  0  1
4  1  0  0
5  0  1  0


In [81]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)

print(df_with_dummy)

   data1  key_a  key_b  key_c
0      0      0      1      0
1      1      0      1      0
2      2      1      0      0
3      3      0      0      1
4      4      1      0      0
5      5      0      1      0
